# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [32]:
# import libraries
import pandas as pd
import nltk
import numpy as np

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [3]:
nltk.download(['punkt','stopwords','wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tati\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tati\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM Messages', engine)

In [5]:
X = df['message']
y = df.drop(columns=['id', 'message', 'original','genre'])

### 2. Write a tokenization function to process your text data

In [6]:
# save stop words in english
stop_english = stopwords.words("english")

In [7]:
def tokenize(text):
    # normalize and tokenize
    tokens = word_tokenize(text.lower())
    
    # remove stop words
    tokens = [word for word in tokens if word not in stop_english]
    
    # lemmatize nouns
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens]
    
    # lemmatize verbs
    tokens = [WordNetLemmatizer().lemmatize(word, pos = 'v') for word in tokens]
    
    # remove trailing space
    tokens = list(map(str.strip, tokens))
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001CA69ACC1F8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.7, random_state=10)

In [22]:
my_model = pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
y_pred = pipeline.predict(X_test)

In [51]:
test_pred = pd.DataFrame(y_pred, columns=list(y.columns))

In [52]:
test_y = y_test.reset_index(drop=True)

In [61]:
for col in list(y.columns):
    print(col)
    print (classification_report(test_y[col], test_pred[col]))

related
              precision    recall  f1-score   support

           0       0.75      0.28      0.40      4297
           1       0.81      0.97      0.88     13913
           2       0.50      0.03      0.05       142

    accuracy                           0.80     18352
   macro avg       0.68      0.43      0.45     18352
weighted avg       0.79      0.80      0.76     18352

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     15211
           1       0.85      0.44      0.58      3141

    accuracy                           0.89     18352
   macro avg       0.87      0.71      0.76     18352
weighted avg       0.89      0.89      0.88     18352

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18277
           1       0.00      0.00      0.00        75

    accuracy                           1.00     18352
   macro avg       0.50      0.50      0.50     1835


weather_related
              precision    recall  f1-score   support

           0       0.88      0.96      0.92     13277
           1       0.85      0.66      0.74      5075

    accuracy                           0.87     18352
   macro avg       0.87      0.81      0.83     18352
weighted avg       0.87      0.87      0.87     18352

floods
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     16835
           1       0.91      0.49      0.63      1517

    accuracy                           0.95     18352
   macro avg       0.93      0.74      0.80     18352
weighted avg       0.95      0.95      0.95     18352

storm
              precision    recall  f1-score   support

           0       0.94      0.99      0.97     16649
           1       0.79      0.43      0.55      1703

    accuracy                           0.94     18352
   macro avg       0.87      0.71      0.76     18352
weighted avg       0.93      0.94      0.93 

In [77]:
acc = (test_pred == test_y).mean().mean()
print('Total Accuracy: {:.4f}'.format(acc))

Total Accuracy: 0.9465


### 6. Improve your model
Use grid search to find better parameters. 
Source: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [81]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {
     'clf__estimator__bootstrap': bootstrap,
     'clf__estimator__max_depth': max_depth,
     'clf__estimator__max_features': max_features,
     'clf__estimator__min_samples_leaf': min_samples_leaf,
     'clf__estimator__min_samples_split': min_samples_split,
     'clf__estimator__n_estimators': n_estimators}

print(random_grid)

{'clf__estimator__bootstrap': [True, False], 'clf__estimator__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'clf__estimator__max_features': ['auto', 'sqrt'], 'clf__estimator__min_samples_leaf': [1, 2, 4], 'clf__estimator__min_samples_split': [2, 5, 10], 'clf__estimator__n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = pipeline, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [65]:
parameters = {
     'clf__estimator__criterion': ['gini', 'entropy'],
     'clf__estimator__min_samples_split': [2,4]
}

cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)
y_pred_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [78]:
test_pred_cv = pd.DataFrame(y_pred_cv, columns=list(y.columns))

for col in list(y.columns):
    print(col)
    print (classification_report(test_y[col], test_pred_cv[col]))

acc_cv = (test_pred_cv == test_y).mean().mean()
print('Total Accuracy: {:.4f}'.format(acc_cv))

related
              precision    recall  f1-score   support

           0       0.73      0.28      0.40      4297
           1       0.81      0.97      0.88     13913
           2       0.50      0.03      0.05       142

    accuracy                           0.80     18352
   macro avg       0.68      0.42      0.44     18352
weighted avg       0.78      0.80      0.76     18352

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.94     15211
           1       0.85      0.43      0.57      3141

    accuracy                           0.89     18352
   macro avg       0.87      0.71      0.75     18352
weighted avg       0.89      0.89      0.87     18352

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18277
           1       0.00      0.00      0.00        75

    accuracy                           1.00     18352
   macro avg       0.50      0.50      0.50     1835

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18264
           1       0.00      0.00      0.00        88

    accuracy                           1.00     18352
   macro avg       0.50      0.50      0.50     18352
weighted avg       0.99      1.00      0.99     18352

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18142
           1       0.00      0.00      0.00       210

    accuracy                           0.99     18352
   macro avg       0.49      0.50      0.50     18352
weighted avg       0.98      0.99      0.98     18352

other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     17560
           1       0.20      0.00      0.00       792

    accuracy                           0.96     18352
   macro avg       0.58      0.50      0.49     18352
weighted avg       0.92      0.96      0.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [67]:
import pickle

In [68]:
# save the model to disk
filename = 'finalized_model.pkl'
pickle.dump(cv, open(filename, 'wb'))

In [71]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

0.23256320836966


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.